In [1]:
import pandas as pd
import numpy as np


In [2]:

# (a) stocks1
stocks1 = pd.read_csv("stocks1.csv")

print(stocks1.head())
print(stocks1.tail())
print(stocks1.dtypes)
print(stocks1.info())

# (b) stocks2
stocks2 = pd.read_csv("stocks2.csv")

print(stocks2.head())
print(stocks2.tail())
print(stocks2.dtypes)
print(stocks2.info())

# (c) companies
companies = pd.read_csv("companies.csv")

print(companies)
print(companies.dtypes)
print(companies.info())


         date symbol  open   high    low  close    volume
0  2024-01-01   AAPL   180  182.0  178.0    181  50000000
1  2024-01-02   AAPL   182  185.0  180.0    184  52000000
2  2024-01-03   GOOG   140  142.0  138.0    141  30000000
3  2024-01-04   MSFT   310  312.0    NaN    311  25000000
4  2024-01-05   MSFT   315    NaN  312.0    314  27000000
         date symbol  open   high    low  close    volume
0  2024-01-01   AAPL   180  182.0  178.0    181  50000000
1  2024-01-02   AAPL   182  185.0  180.0    184  52000000
2  2024-01-03   GOOG   140  142.0  138.0    141  30000000
3  2024-01-04   MSFT   310  312.0    NaN    311  25000000
4  2024-01-05   MSFT   315    NaN  312.0    314  27000000
date       object
symbol     object
open        int64
high      float64
low       float64
close       int64
volume      int64
dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------

Kiểm tra Null + thay thế theo quy tắc


In [5]:

# high Null → thay bằng max(high) theo từng symbol
stocks1["high"] = stocks1.groupby("symbol")["high"].transform(
    lambda x: x.fillna(x.max()))

# low Null → thay bằng min(low) theo từng symbol
stocks1["low"] = stocks1.groupby("symbol")["low"].transform(
    lambda x: x.fillna(x.min()))


Gộp stocks1 + stocks2 theo dòng


In [6]:
stocks = pd.concat([stocks1, stocks2], ignore_index=True)
print(stocks.tail(15))

         date symbol  open   high    low  close    volume
0  2024-01-01   AAPL   180  182.0  178.0    181  50000000
1  2024-01-02   AAPL   182  185.0  180.0    184  52000000
2  2024-01-03   GOOG   140  142.0  138.0    141  30000000
3  2024-01-04   MSFT   310  312.0  312.0    311  25000000
4  2024-01-05   MSFT   315  312.0  312.0    314  27000000
5  2024-01-06   AAPL   184  186.0  182.0    185  53000000
6  2024-01-07   GOOG   142  145.0  140.0    144  32000000
7  2024-01-08   MSFT   314  318.0  312.0    317  29000000


 Gộp stocks với companies


In [7]:
stocks_companies = stocks.merge(companies, left_on="symbol", right_on="name", how="left")
print(stocks_companies.head())



         date symbol  open   high    low  close    volume  name  employees  \
0  2024-01-01   AAPL   180  182.0  178.0    181  50000000  AAPL     161000   
1  2024-01-02   AAPL   182  185.0  180.0    184  52000000  AAPL     161000   
2  2024-01-03   GOOG   140  142.0  138.0    141  30000000  GOOG     190000   
3  2024-01-04   MSFT   310  312.0  312.0    311  25000000  MSFT     221000   
4  2024-01-05   MSFT   315  312.0  312.0    314  27000000  MSFT     221000   

  headquarters_city headquarters_state  
0         Cupertino                 CA  
1         Cupertino                 CA  
2     Mountain View                 CA  
3           Redmond                 WA  
4           Redmond                 WA  


 Giá trung bình (open, high, low, close) và volume của mỗi công ty



In [8]:
print(
    stocks_companies.groupby("symbol")[["open", "high", "low", "close", "volume"]].mean()
)

         open        high    low       close        volume
symbol                                                    
AAPL    182.0  184.333333  180.0  183.333333  5.166667e+07
GOOG    141.0  143.500000  139.0  142.500000  3.100000e+07
MSFT    313.0  314.000000  312.0  314.000000  2.700000e+07


Giá đóng cửa (close) trung bình, max, min theo công ty


In [9]:
print(
    stocks_companies.groupby("symbol")["close"].agg(["mean", "max", "min"])
)

              mean  max  min
symbol                      
AAPL    183.333333  185  181
GOOG    142.500000  144  141
MSFT    314.000000  317  311


 Tạo cột parsed_time từ date (datetime)



In [10]:
stocks_companies["parsed_time"] = pd.to_datetime(stocks_companies["date"])

print(stocks_companies["parsed_time"].dtype)
print(stocks_companies.head())


datetime64[ns]
         date symbol  open   high    low  close    volume  name  employees  \
0  2024-01-01   AAPL   180  182.0  178.0    181  50000000  AAPL     161000   
1  2024-01-02   AAPL   182  185.0  180.0    184  52000000  AAPL     161000   
2  2024-01-03   GOOG   140  142.0  138.0    141  30000000  GOOG     190000   
3  2024-01-04   MSFT   310  312.0  312.0    311  25000000  MSFT     221000   
4  2024-01-05   MSFT   315  312.0  312.0    314  27000000  MSFT     221000   

  headquarters_city headquarters_state parsed_time  
0         Cupertino                 CA  2024-01-01  
1         Cupertino                 CA  2024-01-02  
2     Mountain View                 CA  2024-01-03  
3           Redmond                 WA  2024-01-04  
4           Redmond                 WA  2024-01-05  


 Thêm cột result (up/down)



In [11]:
stocks_companies["result"] = np.where(
    stocks_companies["close"] > stocks_companies["open"],
    "up",
    "down"
)

print(stocks_companies[["symbol", "open", "close", "result"]].head())

  symbol  open  close result
0   AAPL   180    181     up
1   AAPL   182    184     up
2   GOOG   140    141     up
3   MSFT   310    311     up
4   MSFT   315    314   down
